## Chapter Exercises

### Validating numbers into words

Remember the “numbers into words” exercise in Recursion? You’ll be writing tests to validate the functions you wrote.

Fill in the test cases that print question marks. If you think of
additional tests you could perform, add them.

In [ ]:
import Data.List (intersperse)
import Test.Hspec

digitToWord :: Int -> String
digitToWord 0 = "zero"
digitToWord 1 = "one"
digitToWord 2 = "two"
digitToWord 3 = "three"
digitToWord 4 = "four"
digitToWord 5 = "five"
digitToWord 6 = "six"
digitToWord 7 = "seven"
digitToWord 8 = "eight"
digitToWord 9 = "nine"
digitToWord _ = error "invalid digit"

digits :: Int -> [Int]
digits n
  | n < 0 = error "negative number"
  | n < 10 = [n]
  | otherwise = digits (n `div` 10) ++ [n `mod` 10]

wordNumber :: Int -> String
wordNumber n =
  concat $ intersperse "-" digitsWordList
  where
    digitsList = digits n
    digitsWordList = map digitToWord digitsList

main :: IO ()
main = hspec $ do
  describe "digitToWord" $ do
    it "returns zero for 0" $ do
      digitToWord 0 `shouldBe` "zero"
    it "returns one for 1" $ do
      digitToWord 1 `shouldBe` "one"
  describe "digits" $ do
    it "returns [1] for 1" $ do
      digits 1 `shouldBe` [1]
    it "returns [1, 0, 0] for 100" $ do
      digits 100 `shouldBe` [1, 0, 0]
  describe "wordNumber" $ do
    it "one-zero-zero given 100" $ do
      wordNumber 100 `shouldBe` "one-zero-zero"
    it "nine-zero-zero-one for 9001" $ do
      wordNumber 9001 `shouldBe` "nine-zero-zero-one"

### Using QuickCheck

Test some simple arithmetic properties using QuickCheck.

1.
```haskell
half x = x / 2
-- this property should hold
halfIdentity = (*2) . half
```

In [ ]:
half x = x / 2

halfIdentity = (*2) . half

quickCheck (\x -> halfIdentity x == x)

2.
```haskell
import Data.List (sort)

-- for any list you apply sort to
-- this property should hold
listOrdered :: (Ord a) => [a] -> Bool
listOrdered xs =
    snd $ foldr go (Nothing, True) xs
    where 
        go _ status@(_, False) = status
        go y (Nothing, t) = (Just y, t)
        go y (Just x, t) = (Just y, x >= y)
```

In [2]:
import Data.List (sort)

-- for any list you apply sort to
-- this property should hold
listOrdered :: (Ord a) => [a] -> Bool
listOrdered xs =
    snd $ foldr go (Nothing, True) xs
    where 
        go _ status@(_, False) = status
        go y (Nothing, t) = (Just y, t)
        go y (Just x, t) = (Just y, x >= y)

quickCheck (listOrdered . sort)

3. Now we’ll test the associative and commutative properties of addition:

```haskell
plusAssociative x y z =
    x + (y + z) == (x + y) + z

plusCommutative x y =
    x + y == y + x
```

Keep in mind these properties won’t hold for types based on `IEEE-754` floating point numbers, such as Float or Double.

In [ ]:
plusAssociative x y z =
    x + (y + z) == (x + y) + z

plusCommutative x y =
    x + y == y + x

quickCheck plusAssociative
quickCheck plusCommutative

4. Now do the same for multiplication.


In [ ]:
multAssociative x y z =
    x * (y * z) == (x * y) * z

multCommutative x y =
    x * y == y * x

quickCheck multAssociative
quickCheck multCommutative

5. We mentioned in one of the first chapters that there are some laws involving the relationship of quot and rem and div and mod. Write `QuickCheck` tests to prove them.

```
-- quot rem
(quot x y)*y + (rem x y) == x
(div x y)*y + (mod x y) == x
```

In [ ]:
quotRemProp = ((quot x y) * y + (rem x y)) == x

divModProp = ((div x y) * y + (mod x y)) == x

quickCheck quotRemProp
quickCheck divModProp

7. Test that reversing a list twice is the same as the identity of the list:

```haskell
reverse . reverse == id
```

In [3]:
reverseReverseProp x = (reverse . reverse) x == x

quickCheck reverseReverseProp

9. See if these two functions are equal:

```haskell
foldr (:) == (++)
foldr (++) [] == concat
```

> <span style="color: green">**Answer:**</span>
The first one (`foldr (:) == (++)`) is not true consider the output of code below:

In [4]:
foldr (:) [1,2,3] [4,5,6]

[4,5,6,1,2,3]

In [6]:
[1,2,3] ++ [4,5,6]

[1,2,3,4,5,6]

> The second property (`foldr (++) [] == concat`) holds and we can verify it with `QuickCheck`.

In [ ]:
concatFoldProp x = foldr (++) [] x == concat x

quickCheck concatFoldProp

10. Hm. Is that so?
```haskell
f n xs = length (take n xs) == n
```

> <span style="color: green;">**Answer:**</span> No consider the case `take 10 []` which is an empty list with length 0.

11. Finally, this is a fun one. You may remember we had you compose read and show one time to complete a “round trip.” Well, now you can test that it works:
```haskell
f x = (read (show x)) == x
```

In [ ]:
readShowProp x = (read (show x)) == x

quickCheck readShowProp

### Failure

Find out why this property fails.

```haskell
-- for a function
square x = x * x
-- why does this property not hold?
-- Examine the type of sqrt.
squareIdentity = square . sqrt
```

> <span style="color:green">**Answer:**</span> Floating-point numbers in computers are represented using a finite number of bits, which means that they can only approximate the real numbers. This approximation can lead to rounding errors and subtle differences in the values of expressions involving floating-point numbers.
When you raise a number `x` to the power of `2` (`x * x`) and then take the square root of the result, the floating-point arithmetic involved can introduce small numerical errors that prevent the two sides of the equation from being exactly equal.

### Idempotence

Idempotence refers to a property of some functions in which the
result value does not change beyond the initial application. If you
apply the function once, it returns a result, and applying the same
function to that value won’t ever change it. You might think of a list
that you sort: once you sort it, the sorted list will remain the same
after applying the same sorting function to it. It’s already sorted, so
new applications of the sort function won’t change it.
Use QuickCheck and the following helper functions to demonstrate
idempotence for the following:

```haskell
twice f = f . f
fourTimes = twice . twice
```

1. 
```haskell
f x =
    (capitalizeWord x
    == twice capitalizeWord x)
    &&
    (capitalizeWord x
    == fourTimes capitalizeWord x)
```

> <span style="color: green">**Answer:**</span> Trivial!

2.
```haskell
f' x =
    (sort x
    == twice sort x)
    &&
    (sort x
    == fourTimes sort x)
```

> <span style="color: green">**Answer:**</span> Trivial!

### Make a Gen random generator for the datatype


We demonstrated in the chapter how to make Gen generators for
different datatypes. We are so certain you enjoyed that, we are going
to ask you to do it for some new datatypes:

1. Equal probabilities for each.
```haskell
data Fool =
    Fulse
  | Frue
    deriving (Eq, Show)
```

In [ ]:
data Fool
  = Fulse
  | Frue
  deriving (Eq, Show)

instance Arbitrary Fool where
  arbitrary = elements [Fulse, Frue]

2. $\frac{2}{3}$s chance of Fulse, $\frac{1}{3}$ chance of Frue.

In [ ]:
data Fool
  = Fulse
  | Frue
  deriving (Eq, Show)

instance Arbitrary Fool where
  arbitrary = frequency [(2, return Fulse), (1, return Frue)]

### Validating ciphers
As a final exercise, create QuickCheck properties that verify your Cae-
sar and Vigenère ciphers return the same data after encoding and
decoding a string.

> <span style="color: green">**Answer:**</span>: See cipher directory in the current path as this notebook.